In [1]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
from loader import *
from model import *

import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

In [3]:
x_train_dir = '/content/drive/MyDrive/Colab Notebooks/speech denoiser/x_train_noised_speech'
y_train_dir = '/content/drive/MyDrive/Colab Notebooks/speech denoiser/y_train_clean_audio'

shape = (256, 256)

In [4]:
x_train = get_array(dir= x_train_dir, final_shape=shape)
y_train = get_array(dir=y_train_dir, final_shape=shape)

In [6]:
y_train.shape

(7, 256, 256)

In [7]:
# reshaping to make it suitable for training
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1 )
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], y_train.shape[2], 1 )

In [8]:
model = unet(input_shape=x_train.shape[1:])

Downsample block shape is:  (None, 256, 256, 64)
Downsample block shape is:  (None, 128, 128, 128)
Downsample block shape is:  (None, 64, 64, 256)
Downsample block shape is:  (None, 32, 32, 512)
upsample block shape is:  (None, 32, 32, 512)
upsample block shape is:  (None, 64, 64, 256)
upsample block shape is:  (None, 128, 128, 128)
upsample block shape is:  (None, 256, 256, 64)


In [18]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
history = model.fit(
    x_train, y_train,
    batch_size= 3,
    epochs=25,
)

In [30]:
pred = model.predict(x_train)

1/1 [==============================] - 0s 20ms/step


In [31]:
pred.shape

(7, 256, 256, 1)

In [32]:
pred = pred.reshape(pred.shape[0], pred.shape[1], pred.shape[2])
pred.shape

(7, 256, 256)

In [46]:
import librosa
import librosa.display
import IPython.display as ipd
from glob import glob

import soundfile as sf
import pickle

In [35]:
file_path = 'min_max_values.pkl'
with open(file_path, 'rb') as f:
        min_max_values = pickle.load(f)
min_max_values

{'x_train_noised_speech\\p234_001.wav_spec.npy': {'min': -52.605247,
  'max': 27.39475},
 'x_train_noised_speech\\p234_009.wav_spec.npy': {'min': -53.486153,
  'max': 26.513845},
 'x_train_noised_speech\\p234_010.wav_spec.npy': {'min': -49.832893,
  'max': 30.167105},
 'x_train_noised_speech\\p234_012.wav_spec.npy': {'min': -54.21559,
  'max': 25.784407},
 'x_train_noised_speech\\p234_013.wav_spec.npy': {'min': -51.44835,
  'max': 28.551651},
 'x_train_noised_speech\\p234_014.wav_spec.npy': {'min': -52.17472,
  'max': 27.82528},
 'x_train_noised_speech\\p234_015.wav_spec.npy': {'min': -47.565178,
  'max': 32.434822}}

In [43]:
# constants
HOP_LENGTH = 256
MONO = 22050
SAMPLE_RATE = 16000
FRAME_SIZE = 512
OVERLAP_RATIO=0.3

In [39]:
spec_path = 'x_train_noised_speech\p234_001.wav_spec.npy'

In [36]:
# 2. Obtain the corresponding min max value of the loaded spectrogram
def get_min_max_values(file_path, min_max_values):
    min_val = min_max_values[file_path]['min']
    max_val = min_max_values[file_path]['max']
    return min_val, max_val

def denormalize(spectrogram, min_val, max_val):
    denormalized_spectrogram = spectrogram * (max_val - min_val) + min_val
    return denormalized_spectrogram

# 4. Convert spectrogram to audio signal
def spectrogram_to_audio(spectrogram):
    spectrogram = librosa.db_to_amplitude(spectrogram)
    audio_signal = librosa.istft(spectrogram, hop_length=HOP_LENGTH)
    return audio_signal

In [44]:
min_val, max_val = get_min_max_values(spec_path, min_max_values)
denorm_spectrogram =denormalize(pred[0], min_val, max_val)
audio = spectrogram_to_audio(denorm_spectrogram)
audio

array([-1.9023048e-04, -2.3831049e-04, -2.5540704e-04, ...,
        1.2787224e-11, -1.2784799e-11,  1.2783344e-11], dtype=float32)

In [47]:
sf.write('constructed.wav', audio, samplerate= SAMPLE_RATE)
constructed_audio = glob('*.wav')
constructed_audio

['constructed.wav']

In [48]:
ipd.Audio(constructed_audio[0])